<a href="https://colab.research.google.com/github/CPTR295/ML1/blob/main/Custom_Training_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
np.random.seed(42)
tf.random.set_seed(42)
import matplotlib.pyplot as plt


Custom Loss Function

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

In [3]:
X_train_full,X_test,y_train_full,y_test = train_test_split(housing.data,housing.target.reshape(-1,1),random_state=42)
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full,random_state=42)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [6]:
def huber_fn(y_true,y_pred):
  error = y_true - y_pred
  is_small_error = tf.abs(error) < 1
  squared_loss = tf.square(error) / 2
  linear_loss = tf.abs(error) - 0.5
  return tf.where(is_small_error,squared_loss,linear_loss)

In [7]:
X_train.shape

(11610, 8)

In [8]:
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30,activation='selu',kernel_initializer='lecun_normal',input_shape=input_shape),
    keras.layers.Dense(1)
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(loss=huber_fn,optimizer='nadam',metrics=['mae'])

In [10]:
model.fit(X_train_scaled,y_train,epochs=3,validation_data=(X_valid_scaled,y_valid))

Epoch 1/3
363/363 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 1.0500 - mae: 1.4607 - val_loss: 0.2301 - val_mae: 0.5228
Epoch 2/3
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2164 - mae: 0.5145 - val_loss: 0.1889 - val_mae: 0.4762
Epoch 3/3
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.2088 - mae: 0.5030 - val_loss: 0.1909 - val_mae: 0.4751
